## [temproary] Installation (for Google Colab)

In [ ]:
# Only run this cell once and only if you are on Google Colab
# !pip install git+https://github.com/theresa-hefele/eradist.git@dev_library_theresa


# Import Required Libraries
Import the ERADist package and Matplotlib for plotting.

In [ ]:
from ERApy import ERADist, EmpDist, ERACond, ERANataf, ERARosen
from matplotlib import pyplot as plt
import numpy as np

# Create Distribution Object
Instantiate an ERADist object with specified parameters (Normal distribution using MOM with parameters [0,4]).

In [ ]:
dist = ERADist("Normal","MOM",[0,4])

# Generate Random Samples
Use the random method to generate a specified number of samples (10000).

In [ ]:
ns = 10000
xs = dist.random(ns)

# Plot Histogram
Create a histogram of the generated samples using Matplotlib with 50 bins.

In [ ]:
plt.hist(xs, 50)

# Display Plot and Print Message
Display the plot and print a completion message.

In [ ]:
plt.show()

print("End of Code :C")

# Empirical Distribution (EmpDist)
Create an empirical distribution from sample data and demonstrate its methods.

In [ ]:
# Create sample data
data = np.random.normal(0, 1, 1000)
emp_dist = EmpDist(data)

print(f"Empirical distribution mean: {emp_dist.mean():.3f}")
print(f"Empirical distribution std: {emp_dist.std():.3f}")

# Test PDF and CDF
x_test = np.linspace(-3, 3, 100)
pdf_vals = emp_dist.pdf(x_test)
cdf_vals = emp_dist.cdf(x_test)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.hist(data, bins=30, alpha=0.7, density=True, label='Data')
plt.plot(x_test, pdf_vals, 'r-', label='Empirical PDF')
plt.legend()
plt.title('Empirical Distribution')

plt.subplot(1, 3, 2)
plt.plot(x_test, cdf_vals, 'b-')
plt.title('CDF')

# Generate random samples
random_samples = emp_dist.random(1000)
plt.subplot(1, 3, 3)
plt.hist(random_samples, bins=30, alpha=0.7, density=True, label='Random samples')
plt.plot(x_test, pdf_vals, 'r-', label='PDF')
plt.legend()
plt.title('Random Samples')

plt.tight_layout()
plt.show()

# Conditional Distribution (ERACond)
Demonstrate conditional distribution functionality.

In [ ]:
# Create a conditional normal distribution where mean depends on condition
# param is a lambda function that takes condition and returns [mean, std]
cond_dist = ERACond("normal", "PAR", lambda cond: [cond, 1.0])

# Test conditional parameters
condition = 2.0
params = cond_dist.condParam(condition)
print(f"Conditional parameters for condition {condition}: {params}")

# Test conditional PDF and CDF
x_vals = np.linspace(-2, 6, 100)
pdf_vals = [cond_dist.condPDF(x, condition) for x in x_vals]
cdf_vals = [cond_dist.condCDF(x, condition) for x in x_vals]

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(x_vals, pdf_vals)
plt.title(f'Conditional PDF (condition = {condition})')
plt.xlabel('x')
plt.ylabel('PDF')

plt.subplot(1, 2, 2)
plt.plot(x_vals, cdf_vals)
plt.title(f'Conditional CDF (condition = {condition})')
plt.xlabel('x')
plt.ylabel('CDF')

plt.tight_layout()
plt.show()

# Generate conditional random samples
random_conditional = cond_dist.condRandom(condition)
print(f"Random sample from conditional distribution: {random_conditional}")

# Nataf Transformation (ERANataf)
Demonstrate multivariate distribution transformation using Nataf method.

In [ ]:
# Create two marginal distributions
marg1 = ERADist("Normal", "PAR", [0, 1])
marg2 = ERADist("Normal", "PAR", [0, 1])

# Correlation matrix (0.5 correlation between variables)
correlation_matrix = np.array([[1.0, 0.5],
                               [0.5, 1.0]])

# Create Nataf transformation object
nataf = ERANataf([marg1, marg2], correlation_matrix)

print("Nataf transformation created successfully")

# Generate random samples in original space
X_samples = nataf.random(1000)
print(f"Generated {X_samples.shape[1]} samples of {X_samples.shape[0]} variables")

# Transform to standard normal space
U_samples = nataf.X2U(X_samples)
print(f"Transformed to standard normal space: {U_samples.shape}")

# Transform back
X_back = nataf.U2X(U_samples)
print(f"Transformed back: {X_back.shape}")

# Plot the transformations
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.scatter(X_samples[0, :], X_samples[1, :], alpha=0.5)
plt.title('Original Space')
plt.xlabel('X1')
plt.ylabel('X2')
plt.axis('equal')

plt.subplot(1, 3, 2)
plt.scatter(U_samples[0, :], U_samples[1, :], alpha=0.5)
plt.title('Standard Normal Space')
plt.xlabel('U1')
plt.ylabel('U2')
plt.axis('equal')

plt.subplot(1, 3, 3)
plt.scatter(X_back[0, :], X_back[1, :], alpha=0.5, color='red', label='Reconstructed')
plt.scatter(X_samples[0, :], X_samples[1, :], alpha=0.3, color='blue', label='Original')
plt.title('Reconstruction Check')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()
plt.axis('equal')

plt.tight_layout()
plt.show()

# Rosenblatt Transformation (ERARosen)
Demonstrate multivariate distribution transformation using Rosenblatt method with conditional distributions.

In [ ]:
# Create conditional distributions for Rosenblatt transformation
# X1 ~ Normal(0,1)
# X2 | X1 ~ Normal(X1, 0.5)  (mean depends on X1)

dist1 = ERADist("Normal", "PAR", [0, 1])
dist2 = ERACond("normal", "PAR", lambda x1: [x1, 0.5])

# Dependency structure: X2 depends on X1
distributions = [dist1, dist2]
dependencies = [[], [0]]  # X1 has no dependencies, X2 depends on X1 (index 0)

# Create Rosenblatt transformation
rosen = ERARosen(distributions, dependencies)

print("Rosenblatt transformation created successfully")

# Generate random samples
X_samples = rosen.random(1000)
print(f"Generated {X_samples.shape[1]} samples of {X_samples.shape[0]} variables")

# Transform to independent uniform space
U_samples = rosen.X2U(X_samples)
print(f"Transformed to independent uniform space: {U_samples.shape}")

# Transform back
X_back = rosen.U2X(U_samples)
print(f"Transformed back: {X_back.shape}")

# Plot the results
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.scatter(X_samples[0, :], X_samples[1, :], alpha=0.5)
plt.title('Original Space (X1, X2)')
plt.xlabel('X1')
plt.ylabel('X2')

plt.subplot(1, 3, 2)
plt.scatter(U_samples[0, :], U_samples[1, :], alpha=0.5)
plt.title('Independent Uniform Space (U1, U2)')
plt.xlabel('U1')
plt.ylabel('U2')

plt.subplot(1, 3, 3)
plt.scatter(X_back[0, :], X_back[1, :], alpha=0.5, color='red', label='Reconstructed')
plt.scatter(X_samples[0, :], X_samples[1, :], alpha=0.3, color='blue', label='Original')
plt.title('Reconstruction Check')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()

plt.tight_layout()
plt.show()

print("All eradist components demonstrated successfully!")